In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [4]:
from nltk.stem import WordNetLemmatizer,PorterStemmer
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gradv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gradv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
type(newsgroups_train)

sklearn.utils.Bunch

In [6]:
import pandas as pd
df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()])

In [7]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,11304,11305,11306,11307,11308,11309,11310,11311,11312,11313
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,From: guykuo@carson.u.washington.edu (Guy Kuo)...,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,From: jgreen@amber (Joe Green)\nSubject: Re: W...,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,From: dfo@vttoulu.tko.vtt.fi (Foxvog Douglas)\...,From: bmdelane@quads.uchicago.edu (brian manni...,From: bgrubb@dante.nmsu.edu (GRUBB)\nSubject: ...,From: holmes7000@iscsvax.uni.edu\nSubject: WIn...,From: kerr@ux1.cso.uiuc.edu (Stan Kerr)\nSubje...,...,From: Pegasus@aaa.uoregon.edu (Pegasus)\nSubje...,From: shaig@composer.think.com (Shai Guday)\nS...,From: mrj@cs.su.oz.au (Mark James)\nSubject: R...,From: chein@eng.auburn.edu (Tsan Heui)\nSubjec...,From: adrian@ora.COM (Adrian Nye)\nSubject: im...,From: jim.zisfein@factory.com (Jim Zisfein) \n...,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,From: westes@netcom.com (Will Estes)\nSubject:...,From: steve@hcrlgw (Steven Collins)\nSubject: ...,From: gunning@cco.caltech.edu (Kevin J. Gunnin...
1,7,4,4,1,14,16,13,3,2,4,...,19,17,3,6,5,13,4,3,1,8


In [8]:
df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
df.columns = ['text', 'target']

targets = pd.DataFrame( newsgroups_train.target_names)
targets.columns=['title']

ngout = pd.merge(df, targets, left_on='target', right_index=True)

In [9]:
display(ngout)

,text,target,title
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
17,From: CPKJP@vm.cc.latech.edu (Kevin Parker)\nS...,7,rec.autos
29,From: jimf@centerline.com (Jim Frost)\nSubject...,7,rec.autos
56,From: eliot@lanmola.engr.washington.edu (eliot...,7,rec.autos
64,From: sjp@hpuerca.atl.hp.com (Steve Phillips)\...,7,rec.autos
...,...,...,...
11210,From: koontzd@phobos.lrmsc.loral.com (David Ko...,11,sci.crypt
11217,From: schinagl@fstgds15.tu-graz.ac.at (Hermann...,11,sci.crypt
11243,From: brad@clarinet.com (Brad Templeton)\nSubj...,11,sci.crypt
11254,From: amolitor@nmsu.edu (Andrew Molitor)\nSubj...,11,sci.crypt


In [10]:
sdf = spark.createDataFrame(ngout)

In [11]:
display(sdf)

DataFrame[text: string, target: bigint, title: string]

In [12]:
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id,col

In [13]:
sdf=sdf.withColumn("text_sep",split(sdf.text,"\n\n")).select(col('text'), col('target'), col('title'), col('text_sep').getItem(1), col('text_sep').getItem(2)).withColumn("id", monotonically_increasing_id())

display(sdf)

DataFrame[text: string, target: bigint, title: string, text_sep[1]: string, text_sep[2]: string, id: bigint]

In [14]:
sdf.printSchema()

root
 |-- text: string (nullable = true)
 |-- target: long (nullable = true)
 |-- title: string (nullable = true)
 |-- text_sep[1]: string (nullable = true)
 |-- text_sep[2]: string (nullable = true)
 |-- id: long (nullable = false)



In [15]:
temp_table_name = "newsgroup"

sdf.createOrReplaceTempView(temp_table_name)

In [18]:
%load_ext sparksql_magic

In [22]:
%%sparksql
select * from newsgroup

only showing top 20 row(s)


In [25]:
%%sparksql

select count(*) from newsgroup where `text_sep[2]` is null

count(1)
565


In [26]:
%%sparksql

select count(*) from newsgroup where `text_sep[1]` =''

count(1)
262


In [27]:
from pyspark.sql.types import FloatType
import re

def clean_text(in_string):
    remove_email=re.sub('\S*@\S*\s?', '', in_string)
    remove_nl=re.sub('\s+', ' ', remove_email)
    remove_othr = re.sub("\'|\>|\:|\-", "", remove_nl)
    return remove_othr

spark.udf.register("clean", clean_text)

<function __main__.clean_text(in_string)>

In [28]:
%%sparksql
select clean(CASE when`text_sep[2]` is null then `text_sep[1]` when `text_sep[1]`='' then `text_sep[2]` else CONCAT(`text_sep[1]`, ' ', `text_sep[2]`) END), target, title, id FROM newsgroup where `text_sep[2]` is not null and `text_sep[1]` <> ''

only showing top 20 row(s)


"clean(CASE WHEN isnull(text_sep[2]) THEN text_sep[1] WHEN (text_sep[1] = ) THEN text_sep[2] ELSE concat(text_sep[1], , text_sep[2]) END)",target,title,id
"I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please email. Thanks, IL brought to you by your neighborhood Lerxst",7,rec.autos,0
"(Trevor Corkum) writes Is it only me, or is safety not one of the most important factors when buying a car? It depends on your priorities. A lot of people put higher priorities on gas mileage and cost than on safety, buying ""unsafe"" econoboxes instead of Volvos. I personally take a middle ground the only thing I really look for is a threepoint seatbelt and 5+mph bumpers. I figure that 30mph collisions into brick walls arent common enough for me to spend that much extra money for protection, but there are lots of lowspeed collisions that do worry me.",7,rec.autos,2
"an excellent automatic can be found in the subaru legacy. it switches to ""sport"" mode when the electronics figure it, not when the driver sets the switch.. which is the proper way to do it, IMO. so what does ""sport"" mode entail? several things 1) revving to red line (or to the rev limiter in the case of the legacy)",7,rec.autos,3
"Ford and his automobile. I need information on whether Ford is partially responsible for all of the car accidents and the depletion of the ozone layer. Also, any other additional information will be greatly appreciated. Thanks. SSSSSoooooooooooo!!!!! Its all HIS fault!! Thank God Louis Chevrolet is innocent! and that guy Diesel, HE otto feel guilty! Stephen Phillips Atlanta Response Center Atlanta, Ga. Home of the Braves!",7,rec.autos,4
"In article (Matthew MacIntyre at the National University of Senegal) writes (James P. Callison) writes Im not going to argue the issue of carrying weapons, but I would ask you if you would have thought seriously about shooting a kid for setting off your alarm? I can think of worse things in the world. Glad you got out of there before they did anything to give you a reason to fire your gun. I think people have a right to kill to defend their property. Why not? Be honest do you really care more about scum than about your car? Yo! Watch the attributionsI didnt say that!",7,rec.autos,5
"In article (Aviad Sheinfeld) writes Do you think I can use a electric drill( change to a suitable bit ) to turn it out? If I can succeed, can I retighten it not too tight, is it safe without oil leak? Tighten the bolt to the specified torque in your service manual. That way it wont leak, strip, break, etc. (hopefully ) ) Thank you very much in advance Winson Aviad You can avoid these problems entirely by installing an oil drain valve in place of the bolt. I have one on both of my cars. There have been no leaks in 210,000 miles (combined miles on both cars). Ron DeBlock (thats a number 1 in rdb1, not letter l) AT&T Bell Labs Somerset, NJ USA",7,rec.autos,6
"I have a 1986 Acura Integra 5 speed with 95,000 miles on it. It is positively the worst car I have ever owned. I had an 83 Prelude that had 160k miles on it when I sold it, and it was still going strong . This is with religious attention to maintenance such as oil changes etc. Both cars were driven in exactly the same manner.. 1. It has gone through two clutches (which are underrated.) 2. 3 sets of tires (really eats tires in the front even with careful align) 3. All struts started leaking about 2530k miles 4. Windshield wiper motor burned up (service note on this one) 5. Seek stop working on radio about 20k miles 6. Two timing belts. 7. Constant error signals from computer.",7,rec.au

In [29]:
sdf=spark.sql("select clean(CASE when`text_sep[2]` is null then `text_sep[1]` when `text_sep[1]`='' then `text_sep[2]` else CONCAT(`text_sep[1]`, ' ', `text_sep[2]`) END) as text, target, title, id FROM newsgroup where `text_sep[2]` is not null and `text_sep[1]` <> ''")

In [30]:
display(sdf)

DataFrame[text: string, target: bigint, title: string, id: bigint]

In [31]:
sdf.count()

10490

In [33]:
from pyspark.sql.functions import col, length
sdf.where(length(col('text')) < 100)

DataFrame[text: string, target: bigint, title: string, id: bigint]

In [34]:
sdf=sdf.where(length(col('text')) > 100)

In [35]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
tokenizer = RegexTokenizer(inputCol="text", outputCol="tokens", pattern="\\W+", minTokenLength=4, toLowercase=True)
tokenized = tokenizer.transform(sdf)

In [36]:
display(tokenized)

DataFrame[text: string, target: bigint, title: string, id: bigint, tokens: array<string>]

In [37]:
spremover = StopWordsRemover(inputCol="tokens", outputCol="spfiltered")
spremoved = spremover.transform(tokenized)

display(spremoved.select("tokens","spfiltered"))

DataFrame[tokens: array<string>, spfiltered: array<string>]

In [38]:
porter = PorterStemmer()
lemma = WordNetLemmatizer()
def word_tokenize(text):
    #print(text)
    pos = nltk.pos_tag(text)
    final = [lemma.lemmatize(word[0]) if (lemma.lemmatize(word[0]).endswith(('e','ion')) or len(word[0]) < 4 ) else porter.stem(word[0]) for word in pos]
    return final

In [39]:
spremoved.printSchema()

root
 |-- text: string (nullable = true)
 |-- target: long (nullable = true)
 |-- title: string (nullable = true)
 |-- id: long (nullable = false)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- spfiltered: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [40]:
#spremoved.withColumn("stemmed",[tokenize(words) for words in spremoved
stemmed = spremoved.rdd.map(lambda tup: (tup[1],tup[2],tup[3], word_tokenize(tup[5])))

In [41]:
stemmed.collect()

[(7,
  'rec.autos',
  0,
  ['wonder',
   'anyone',
   'enlighten',
   '2door',
   'sport',
   'look',
   'late',
   'earli',
   'call',
   'bricklin',
   'door',
   'realli',
   'small',
   'addition',
   'front',
   'bumper',
   'separate',
   'rest',
   'bodi',
   'know',
   'anyone',
   'tellme',
   'model',
   'name',
   'engine',
   'spec',
   'year',
   'production',
   'made',
   'histori',
   'whatev',
   'info',
   'funki',
   'look',
   'please',
   'email',
   'thank',
   'brought',
   'neighborhood',
   'lerxst']),
 (7,
  'rec.autos',
  2,
  ['trevor',
   'corkum',
   'write',
   'safeti',
   'import',
   'factor',
   'buy',
   'depend',
   'prioriti',
   'people',
   'higher',
   'prioriti',
   'mileage',
   'cost',
   'safeti',
   'buy',
   'unsafe',
   'econobox',
   'instead',
   'volvo',
   'person',
   'take',
   'middle',
   'ground',
   'thing',
   'realli',
   'look',
   'threepoint',
   'seatbelt',
   'bumper',
   'figure',
   '30mph',
   'collision',
   'brick',


In [42]:
news_df=stemmed.toDF(schema=['target','title','id','word'])

In [43]:
display(news_df)

DataFrame[target: bigint, title: string, id: bigint, word: array<string>]

In [44]:
spwordlist = ["article", "write", "entry", "date", "udel", "said", "tell", "think", "know", "just", "isnt", "line", "like", "does", "going", "make", "thanks","also"]

spremover1 = StopWordsRemover(inputCol="word", outputCol="word_new", stopWords=spwordlist)
news_df = spremover1.transform(news_df)

display(news_df.select("word","word_new"))

DataFrame[word: array<string>, word_new: array<string>]

In [45]:
display(news_df)

DataFrame[target: bigint, title: string, id: bigint, word: array<string>, word_new: array<string>]

In [46]:
df_explode = news_df.withColumn('word_new', explode('word_new'))

In [47]:
display(df_explode)

DataFrame[target: bigint, title: string, id: bigint, word: array<string>, word_new: string]

In [48]:
df_explode.createOrReplaceTempView("topwords")

In [49]:
%%sparksql
select word_new, count(*) as freq from topwords group by word_new order by freq desc

only showing top 20 row(s)


word_new,freq
dont,1989
use,1905
maxaxaxaxaxaxaxaxaxaxaxaxaxaxax,1888
people,1773
time,1648
work,1440
anyone,1322
look,1306
good,1279
year,1262


In [50]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="word_new", outputCol="rawFeatures", vocabSize = 10000, minDF=5)
cvmodel = cv.fit(news_df)
featurizedData = cvmodel.transform(news_df)

In [51]:
display(featurizedData)

DataFrame[target: bigint, title: string, id: bigint, word: array<string>, word_new: array<string>, rawFeatures: vector]

In [66]:
vocab = cvmodel.vocabulary
vocab_broadcast = spark.sparkContext.broadcast(vocab)
vocab

['dont',
 'use',
 'maxaxaxaxaxaxaxaxaxaxaxaxaxaxax',
 'people',
 'time',
 'work',
 'anyone',
 'look',
 'good',
 'year',
 'want',
 'window',
 'problem',
 'need',
 'well',
 'thing',
 'post',
 'right',
 'even',
 'system',
 'question',
 'much',
 'first',
 'drive',
 'seem',
 'come',
 'call',
 'mani',
 'thank',
 'take',
 'game',
 'please',
 'point',
 'realli',
 'read',
 'file',
 'back',
 'someth',
 'find',
 'help',
 'since',
 'say',
 'mean',
 'last',
 'program',
 'number',
 'card',
 'start',
 'interest',
 'tri',
 'someone',
 'go',
 'wrote',
 'cant',
 'give',
 'state',
 'sure',
 'believe',
 'still',
 'follow',
 'never',
 'better',
 'might',
 'person',
 'anoth',
 'christian',
 'part',
 'case',
 'doesnt',
 'team',
 'power',
 'group',
 'control',
 'actual',
 'name',
 'run',
 'get',
 'email',
 'chip',
 'little',
 'without',
 'information',
 'comput',
 'everi',
 'made',
 'anyth',
 'play',
 'must',
 'govern',
 'that',
 'probabl',
 'didnt',
 'reason',
 'around',
 'fact',
 'happen',
 'driver',
 'diff

In [67]:
from pyspark.ml.feature import IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [68]:
display(rescaledData)

DataFrame[target: bigint, title: string, id: bigint, word: array<string>, word_new: array<string>, rawFeatures: vector, features: vector]

In [69]:
rescaledData.printSchema()

root
 |-- target: long (nullable = true)
 |-- title: string (nullable = true)
 |-- id: long (nullable = true)
 |-- word: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word_new: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)



In [70]:
corpus = rescaledData.select('id','features').cache()
display(corpus)

DataFrame[id: bigint, features: vector]

In [71]:
from pyspark.ml.clustering import LDA

lda = LDA(k=20, maxIter=50, optimizer="em")
model = lda.fit(corpus)

In [72]:
ll = model.logLikelihood(corpus)
lp = model.logPerplexity(corpus)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))


The lower bound on the log likelihood of the entire corpus: -46331553.49901123
The upper bound on perplexity: 28.283477018621646


In [73]:
topicwords=20
topics = model.describeTopics(topicwords)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

The topics described by their top-weighted terms:
+-----+-----------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                                                                                      |termWeights                                                                                                                                                                                                                                                         

In [74]:
tRDD = topics.rdd.map(list)

In [75]:
topics.printSchema()

root
 |-- topic: integer (nullable = false)
 |-- termIndices: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- termWeights: array (nullable = true)
 |    |-- element: double (containsNull = false)



In [76]:
tRDD.collect()

[[0,
  [88,
   78,
   405,
   439,
   456,
   3,
   369,
   19,
   510,
   17,
   594,
   762,
   190,
   642,
   513,
   518,
   45,
   794,
   491,
   604],
  [0.010361937444035676,
   0.009857472884826034,
   0.008413255351732263,
   0.008162011487199434,
   0.008113578114235498,
   0.007317679982638066,
   0.006908737664819941,
   0.006211355124941302,
   0.006168032987168963,
   0.005926499833825332,
   0.005897917362769375,
   0.005610413175502949,
   0.005493626628085393,
   0.0054721232650365315,
   0.0052349935610809875,
   0.004877133696657338,
   0.004847398926044883,
   0.004763920924673321,
   0.004721658165140238,
   0.004607389381666725]],
 [1,
  [312,
   495,
   265,
   450,
   407,
   406,
   114,
   620,
   361,
   538,
   1104,
   931,
   5,
   444,
   874,
   825,
   840,
   1172,
   7,
   1064],
  [0.009811463985262482,
   0.009244115900182683,
   0.008933449806511217,
   0.008305790457755808,
   0.008204668956915986,
   0.00751033772367417,
   0.007379652232655839

In [77]:
def topic_vocab(topic):
    print(topic)
    topicNum = topic[0]
    terms = topic[1]
    weight = topic[2]
    result = []
    for i in range(topicwords):
        term = vocab[terms[i]]
        out = str(topicNum)+","+term+","+str(weight[i])
        result.append(out)
    return result

In [78]:
tRDD.map(lambda topic: topic_vocab(topic)).collect()

[['0,govern,0.010361937444035676',
  '0,chip,0.009857472884826034',
  '0,clipper,0.008413255351732263',
  '0,key,0.008162011487199434',
  '0,encryption,0.008113578114235498',
  '0,people,0.007317679982638066',
  '0,legal,0.006908737664819941',
  '0,system,0.006211355124941302',
  '0,protect,0.006168032987168963',
  '0,right,0.005926499833825332',
  '0,citizen,0.005897917362769375',
  '0,escrow,0.005610413175502949',
  '0,phone,0.005493626628085393',
  '0,secur,0.0054721232650365315',
  '0,law,0.0052349935610809875',
  '0,appli,0.004877133696657338',
  '0,number,0.004847398926044883',
  '0,amend,0.004763920924673321',
  '0,million,0.004721658165140238',
  '0,algorithm,0.004607389381666725'],
 ['1,car,0.009811463985262482',
  '1,tape,0.009244115900182683',
  '1,light,0.008933449806511217',
  '1,condition,0.008305790457755808',
  '1,front,0.008204668956915986',
  '1,trade,0.00751033772367417',
  '1,enough,0.0073796522326558395',
  '1,sport,0.007060373798173572',
  '1,jame,0.00617301247256

In [79]:
topic = tRDD.map(lambda topic: topic_vocab(topic))

In [80]:
ng_pd = spark.createDataFrame(topic).toPandas()

In [81]:
ng_pd.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
_1,"0,govern,0.010361937444035676","1,car,0.009811463985262482","2,game,0.02223937304014027","3,space,0.020041461148904514","4,drive,0.02859592478165273","5,cause,0.0089268201520493","6,moral,0.009279916382538958","7,please,0.014682426306732346","8,monitor,0.016778580349076772","9,appear,0.01639248742459177","10,mail,0.014136257085277366","11,isra,0.01130313304028852","12,told,0.007232100118961025","13,dont,0.008970993813121803","14,christian,0.015183272395317339","15,kill,0.00964522721792859","16,maxaxaxaxaxaxaxaxaxaxaxaxaxaxax,0.337140306...","17,group,0.012356490090546693","18,bike,0.017496139669105733","19,window,0.03333755862576032"
_2,"0,chip,0.009857472884826034","1,tape,0.009244115900182683","2,team,0.01665112483987548","3,station,0.007217982470544751","4,driver,0.016846036446464867","5,insurance,0.008478541777869969","6,truth,0.008589327059297927","7,price,0.013161349643649211","8,color,0.014338747039374499","9,ground,0.01133650598575319","10,compani,0.013148240871886877","11,center,0.011161709396681442","12,month,0.006416409890359874","13,rule,0.00847757291757512","14,jesu,0.012406877362083488","15,greek,0.006934210781256035","16,cool,0.011680886957367695","17,bill,0.012044786057618558","18,ride,0.009639639933830492","19,file,0.02363908446096226"
_3,"0,clipper,0.008413255351732263","1,light,0.008933449806511217","2,play,0.01438853029555865","3,orbit,0.006772667269421179","4,system,0.014121653661628587","5,food,0.00842842563866219","6,human,0.007902122386570077","7,thank,0.013001860459316325","8,mode,0.010263654497404637","9,black,0.01069332445218523","10,address,0.010614152689349409","11,picture,0.011029131104127428","12,effect,0.006273096683461116","13,come,0.008327272599912814","14,church,0.008184246011108247","15,attack,0.006067784776515477","16,water,0.010235554400755615","17,clinton,0.010049437236314593","18,back,0.008663711894626266","19,program,0.020180220327359916"
_4,"0,key,0.008162011487199434","1,condition,0.008305790457755808","2,player,0.012524702723204556","3,launch,0.006762667557324008","4,card,0.013830452228395301","5,thing,0.007988652221494034","6,believe,0.007480753455092209","7,data,0.012913580881932223","8,move,0.009867115222328967","9,power,0.009710127859998773","10,product,0.009641856045246004","11,research,0.010975151482168844","12,office,0.0059676720111850375","13,reason,0.007805643949050451","14,christ,0.007341022945677303","15,people,0.006011680508154772","16,roger,0.00933485162765188","17,cost,0.009643707363563552","18,motorcycle,0.007403165214466202","19,graphic,0.010706626730846087"
_5,"0,encryption,0.008113578114235498","1,front,0.008204668956915986","2,year,0.0089917671307671","3,april,0.006742640639168425","4,disk,0.013356126465834544","5,disease,0.0074683723379659036","6,exist,0.007401545322391872","7,info,0.011717235284292846","8,support,0.009221579937287309","9,cover,0.008919418824037587","10,list,0.009485951372835751","11,arab,0.010143474722558726","12,day,0.005855975046231474","13,event,0.0070967553674689526","14,bible,0.007123833747498755","15,countri,0.0059397782756251466","16,nuclear,0.008481288596897627","17,newsgroup,0.009337752860977993","18,pain,0.006909818417489374","19,application,0.009627765079900971"
_6,"0,people,0.007317679982638066","1,trade,0.00751033772367417","2,season,0.008340545611493262","3,earth,0.005951515201303002","4,mouse,0.01083143443419489","5,shoot,0.005929158513869293","6,atheist,0.007007751061911077","7,information,0.010986982385566184","8,screen,0.008496527558753698","9,wire,0.008503874698548242","10,internet,0.00889225742667477","11,polici,0.008478080181382464","12,drug,0.005849515322180552","13,person,0.007059184833224797","14,people,0.006502501272658591","15,civilian,0.005652336752756469","16,mb8f,0.007659681691925723","17,firearm,0.007945202585628275","18,well,0.006865013969974981","19,use,0.009401780910224538"
_7,"0,legal,0.006908737664819941","1,enough,0.0073796522326558395","2,basebal,0.0076751